In [2]:
import pandas as pd
import numpy as np

In [3]:
contract = pd.read_csv('contract.csv')
internet = pd.read_csv('internet.csv')
personal = pd.read_csv('personal.csv')
phone = pd.read_csv('phone.csv')

### Shape & Info of DataFrames

In [13]:
for df in [contract, personal, internet, phone]:
    print(df.shape)
    print(df.info())
    print('-'*50)


(7043, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB
None
--------------------------------------------------
(7043, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitiz

### Observations so far

- contract.csv → 7,043 rows, 8 columns
- TotalCharges is stored as object, even though it should be numeric (needs conversion).
- personal.csv → 7,043 rows, 5 columns
- internet.csv → 5,517 rows, 8 columns (fewer rows → not every customer has internet service).
- phone.csv → 6,361 rows, 2 columns (again fewer rows → not every customer has phone service).
- No null values reported — at least in .info(). 👍



In [ ]:
# Checking for target count in contract dataframe
contract['EndDate'].value_counts()

EndDate
No                     5174
2019-11-01 00:00:00     485
2019-12-01 00:00:00     466
2020-01-01 00:00:00     460
2019-10-01 00:00:00     458
Name: count, dtype: int64

In [16]:
for name, df in zip(['contract','personal','internet','phone'],
                    [contract, personal, internet, phone]):
    print(f"{name}: {df.duplicated().sum()} duplicate rows")
    print(f"{name}: {df['customerID'].duplicated().sum()} duplicate customerIDs\n")
# Checking for duplicate rows and customerIDs

contract: 0 duplicate rows
contract: 0 duplicate customerIDs

personal: 0 duplicate rows
personal: 0 duplicate customerIDs

internet: 0 duplicate rows
internet: 0 duplicate customerIDs

phone: 0 duplicate rows
phone: 0 duplicate customerIDs



In [17]:
contract[contract['TotalCharges'].str.strip() == ""].shape

(11, 8)

## 🛠 Planned Workflow

### 1. Data Preparation
- Merge all datasets using `customerID`.
- Process target:  
  - `EndDate = "No"` → `0` (active)  
  - `EndDate = date` → `1` (churn)  
- Convert `TotalCharges` to numeric. Handle 11 blank values (impute as `0` or drop).  
- Fill missing service data: if a customer has no row in `internet` or `phone`, mark features as `"No"`.  
- Encode categorical variables (binary → 0/1; multi-class → one-hot or similar).  

### 2. Exploratory Data Analysis (EDA)
- Study churn distribution across:  
  - Contract type (month-to-month, 1-year, 2-year)  
  - Payment method  
  - Internet service type  
  - Additional services (OnlineSecurity, Backup, Streaming, etc.)  
- Visualize churn by key features.  
- Check correlations between features.  

### 3. Feature Engineering
- Create `tenure` = difference between `BeginDate` and `EndDate` (or Feb 1, 2020, if active).  
- Create `service_count` = number of services subscribed.  
- Transform categorical features into numerical encodings.  
- Add contract/payment features (e.g., flag auto-pay customers).  

### 4. Modeling Approach
- Split data into **train/validation/test** (stratified).  
- Train baseline models: Logistic Regression, Decision Tree, Random Forest.  
- Train advanced models: Gradient Boosting (LightGBM, CatBoost, XGBoost).  
- Handle class imbalance with class weights or resampling techniques.  

### 5. Evaluation
- **Primary metric:** AUC-ROC.  
- **Secondary metric:** Accuracy (will also check Precision/Recall due to imbalance).  
- Compare models and select best-performing one.  

### 6. Reporting
- Document preprocessing steps, EDA insights, and model results.  
- Summarize key findings and **business insights** (e.g., which features influence churn).  
- Provide recommendations for how Interconnect can reduce churn.  


## ❓ Clarifying Questions for Team Lead

1. **Target Definition**  
   Just to confirm — for the churn target, should we treat `EndDate = "No"` as **active (0)** and any date value as **churned (1)**?

2. **Blank `TotalCharges`**  
   We identified **11 rows** where `TotalCharges` is blank. Should these be imputed as `0` (new customers with no billing yet), or excluded from the dataset?

3. **Evaluation Metrics**  
   The requirement states **AUC-ROC** as the primary metric and **Accuracy** as secondary. Since the dataset is imbalanced (~26.5% churn), would it make sense to also include **Precision/Recall** (or F1-score) to better reflect model performance on churned customers?
